In [1]:
from datasets import Dataset
from transformers import AutoModel,AutoTokenizer,Trainer,TrainingArguments,DataCollatorForSeq2Seq,AutoModelForCausalLM
import datasets
import torch
import pandas as pd

/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [28]:
# ds = pd.read_json('/home/ys/zhy/chatglm微调/chat-dataset-baseline-main/history/其他中文问题补充/三国问题.json')
ds = pd.read_json('./alpaca/alpaca_gpt4_data_zh.json')

In [29]:
ds

,instruction,input,output
0,保持健康的三个提示。,,以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游...
1,三原色是什么？,,三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在...
2,描述原子的结构。,,原子是物质的基本单位，它由三种基本粒子组成：质子、中子和电子。质子和中子形成原子核，位于原子...
3,如何减少空气污染？,,有很多方法来减少空气污染:\n\n1. 减少机动车排放: 通过支持可再生能源来驱动的交通工具...
4,描述一次你不得不做出困难决定的经历。,,作为一名人工智能助手，我是由程序控制的，没有感情和个人意识，我的一切决策都是基于我的算法和程...
...,...,...,...
48813,生成一份烹饪专业人士的简历示例。,,以下是一份烹饪专业人士的简历示例：\n\n个人信息\n姓名: 王大厨 \n联系电话: 123...
48814,蛋糕，我，吃,,好的，你可以去吃蛋糕。享受美食！
48815,写一段关于著名人物米歇尔·奥巴马的介绍。,,米歇尔·奥巴马是一位杰出的律师、公益活动家和作家，她曾在2009年到2017年间担任美国第一...
48816,考虑职业转换时应记住的五件事的清单。,,当考虑进行职业转换时，以下是值得记住的五件事情：\n\n1. 研究新职业: 在转型到新的职业...


In [30]:
ds = Dataset.from_pandas(ds)

In [31]:
ds[:3]

{'instruction': ['保持健康的三个提示。', '三原色是什么？', '描述原子的结构。'],
 'input': ['', '', ''],
 'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。',
  '原子是物质的基本单位，它由三种基本粒子组成：质子、中子和电子。质子和中子形成原子核，位于原子中心，核外的电子围绕着原子核运动。\n\n原子结构具有层次性。原子核中，质子带正电，中子不带电（中性）。原子核非常小且致密，占据了原子总质量的绝大部分。电子带负电，通常围绕核运动，形成若干层次，称为壳层或电子层。电子数量与质子数量相等，使原子呈电中性。\n\n电子在每个壳层中都呈规律分布，并且不同壳层所能容纳的电子数也不同。在最里面的壳层一般只能容纳2个电子，其次一层最多可容纳8个电子，再往外的壳层可容纳的电子数逐层递增。\n\n原子核主要受到两种相互作用力的影响：强力和电磁力。强力的作用范围非常小，主要限制在原子核内，具有极强的吸引作用，使核子（质子和中子）紧密结合在一起。电磁力的作用范围较大，主要通过核外的电子与原子核相互作用，发挥作用。\n\n这就是原子的基本结构。原子内部结构复杂多样，不同元素的原子核中质子、中子数量不同

In [32]:
model_name_Path = './bloom1b4/'

In [49]:
tokenizer = AutoTokenizer.from_pretrained(model_name_Path,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name_Path,low_cpu_mem_usage = True,load_in_8bit = False,trust_remote_code=True)

In [8]:
inputs = tokenizer('政治这个词是什么意思?请详细解释',truncation=True,max_length=512,return_tensors='pt')
output = model.generate(input_ids = inputs['input_ids'],)
assert 0

/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


AssertionError: 

In [ ]:
# tokenizer.decode(output[0])


In [9]:
tokenizer.decode(output[0])

'政治这个词是什么意思?请详细解释一下。</s>'

In [10]:
for name,param in model.named_parameters():
    print(name,param.shape,param.dtype)

transformer.word_embeddings.weight torch.Size([46145, 2048]) torch.float16
transformer.word_embeddings_layernorm.weight torch.Size([2048]) torch.float16
transformer.word_embeddings_layernorm.bias torch.Size([2048]) torch.float16
transformer.h.0.input_layernorm.weight torch.Size([2048]) torch.float16
transformer.h.0.input_layernorm.bias torch.Size([2048]) torch.float16
transformer.h.0.self_attention.query_key_value.weight torch.Size([6144, 2048]) torch.int8
transformer.h.0.self_attention.query_key_value.bias torch.Size([6144]) torch.float16
transformer.h.0.self_attention.dense.weight torch.Size([2048, 2048]) torch.int8
transformer.h.0.self_attention.dense.bias torch.Size([2048]) torch.float16
transformer.h.0.post_attention_layernorm.weight torch.Size([2048]) torch.float16
transformer.h.0.post_attention_layernorm.bias torch.Size([2048]) torch.float16
transformer.h.0.mlp.dense_h_to_4h.weight torch.Size([8192, 2048]) torch.int8
transformer.h.0.mlp.dense_h_to_4h.bias torch.Size([8192]) torc

In [33]:
# 数据集处理
max_seq_len = 512
def preprocess(example):
    context_id = tokenizer.encode(example['instruction'],truncation = True,max_length=max_seq_len)
    target_id = tokenizer.encode(example['output'],truncation = True,max_length=max_seq_len,add_special_tokens = False)
    input_ids = context_id + target_id + [tokenizer.eos_token_id]
    return {'input_ids':input_ids,'context_len':len(context_id)}
def data_collator(features:list):
    len_id = [len(x['input_ids']) for x in features]
    longest = max(len_id)
    input_ids = []
    label_ls = []
    for length,feature in sorted(zip(len_id,features),key = lambda x:-x[0]):  # 按照batch中最长的input_ids进行padding
        ids = feature['input_ids']
        context_len = feature['context_len']

        label = (
            [-100]*(context_len - 1) + ids[(context_len - 1):] + [-100]*(longest-length) #  (instruct - > -100)   +target + padding
        )
        ids = ids + [tokenizer.pad_token_id] *(longest-length)
        label_ls.append(torch.LongTensor(label))
        input_ids.append(torch.LongTensor(ids))
    input_ids = torch.stack(input_ids)
    labels = torch.stack(label_ls)
    return {'input_ids':input_ids,'labels':labels}

In [12]:
# 没有attention_mask的
# max_seq_len = 512
# def precess(example):
#     context_id = tokenizer.encode(example['instruction'],max_length = max_seq_len,truncation = True)
#     target_id = tokenizer.encode(example['output'],max_length = max_seq_len,add_special_tokens = False,truncation = True)
#     input_ids = context_id + target_id + [tokenizer.eos_token_id]
#     return {'input_ids':input_ids,'context_len':len(context_id),'target_len':len(target_id)}
# def data_collator(features:list):
#     len_id = [len(x['input_ids']) for x in features]
#     longest = max(len_id)
#     input_ids = []
#     label_ls = []
#     for length,feature in sorted(zip(len_id,features),key = lambda x:-x[0]):  # 按照batch中最长的input_ids进行padding
#         ids = feature['input_ids']
#         context_len = feature['context_len']

#         label = (
#             [-100]*(context_len - 1) + ids[(context_len - 1):] + [-100]*(longest-length) #  (instruct - > -100)   +target + padding
#         )
#         ids = ids + [tokenizer.pad_token_id] *(longest-length)
#         label_ls.append(torch.tensor(label))
#         input_ids.append(torch.tensor(ids))
#     input_ids = torch.stack(input_ids)
#     # print(input_ids)
#     labels = torch.stack(label_ls)
#     return {'input_ids':input_ids,'labels':labels}
    

In [34]:
inputs

{'input_ids': tensor([[ 3783, 39155, 38495,    34,  2680, 11518,  9047]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [35]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [36]:
tokenizer_ds  = ds.map(process_func,remove_columns=ds.column_names)
tokenizer_ds

Map:   0%|          | 0/48818 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 48818
})

In [37]:
tokenizer.decode(tokenizer_ds[1]["input_ids"])

'Human: 三原色是什么？\n\nAssistant: 三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。</s>'

### Ptuning step 1 配置文件


In [50]:
from peft import PromptEncoderConfig, TaskType, get_peft_model, PromptEncoderReparameterizationType

config = PromptEncoderConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10,
                             encoder_reparameterization_type=PromptEncoderReparameterizationType.MLP,
                             encoder_dropout=0.1, encoder_num_layers=5, encoder_hidden_size=1024)
config

PromptEncoderConfig(peft_type=<PeftType.P_TUNING: 'P_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, encoder_reparameterization_type=<PromptEncoderReparameterizationType.MLP: 'MLP'>, encoder_hidden_size=1024, encoder_num_layers=5, encoder_dropout=0.1)

In [51]:
model = get_peft_model(model,config)

/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/peft/tuners/p_tuning/model.py:106: UserWarning: for MLP, the argument `encoder_num_layers` is ignored. Exactly 2 MLP layers are used.
  warnings.warn(


In [52]:
# model.supports_gradient_checkpointing = True
# model.gradient_checkpointing_enable()
# model.enable_input_require_grads()


In [53]:
model.cuda()

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [54]:
model.print_trainable_parameters()

trainable params: 5,267,456 || all params: 1,308,379,136 || trainable%: 0.4025940077356905


In [55]:
# 创建训练文件
args = TrainingArguments(
    output_dir = './',
    per_device_train_batch_size = 2,
    num_train_epochs =1,
    weight_decay = 0.1,
    gradient_accumulation_steps=4,
    lr_scheduler_type = 'linear',
    learning_rate = 1e-5,
    fp16 = True,
    remove_unused_columns=True,
    logging_steps=1,
    
)
# 创建训练器
trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenizer_ds,
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True),
    # data_collator=data_collator,
    args = args,
    
)
trainer.train()

/home/ys/anaconda3/envs/chatglm_test/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.281700
2,2.341800
3,2.876200
4,2.939000
5,2.673500
6,2.677000
7,2.767300
8,2.410000
9,3.296000
10,2.232900


KeyboardInterrupt: 

In [ ]:
inputs = tokenizer('赵云是怎么死的？',return_tensors='pt').to(model.device)
#inputs[0].numpy().tolist()
inputs

In [ ]:
output = model.generate(**inputs)

In [ ]:
output

In [ ]:
#output

In [ ]:
tokenizer.decode(output[0].cpu().numpy().tolist(),skip_special_tokens = True,max_length = 512)

In [ ]:
model = model.merge_and_unload() 

In [ ]:
model.generate(inputs.to('cuda'))

In [ ]:
model.merge_and_upload()

In [ ]:
# dl = torch.utils.data.DataLoader(tokenizer_ds)

In [ ]:
# torch.tensor(batch['input_ids']).to('cuda').shape,torch.tensor(batch['attention_mask']).to('cuda').shape


In [ ]:

for batch in dl:
    x = model(torch.tensor(batch['input_ids']).to('cuda'),attention_mask = torch.tensor(batch['attention_mask']).to('cuda'))
    break

In [ ]:
batch['input_ids'].shape

In [ ]:
help(model)


In [ ]:
tokenizer(['我爱不'])

In [ ]:
h